In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import os
import sys
from tqdm import tqdm

In [ ]:
# Load the model
model = tf.keras.models.load_model('models/test25.h5')

image_files = os.listdir('tub_1_22-11-12/images/')
images = np.zeros((len(image_files),120,160,3), dtype="float")
true_angles_all = np.zeros((len(image_files),1))
lr = 0.5

In [ ]:
for count in tqdm(range(len(image_files)-len(image_files)%1000)):
    # Read in the new catalog. From catalog we get true values for the picture.
    if count % 1000 == 0:
        with open(f'tub_1_22-11-12/catalog_{count%1000}.catalog', 'r') as f:
            text_org = f.read()
            text = text_org.split("\n")
        with open(f'images_adversarial/tub_1_22-11-12_adversarial/catalog_{int(count/1000)}.catalog', 'w') as f:
            f.write(text_org)
    
    # Get the true throttle and steering angle of the image.
    values = text[count%1000].split(", ")
    filename = values[3][20:-1]
    true_angles_all[count] = float(values[4][14:])
    #true_throttle = float(values[6][17:-1])
    
    # Load in the image
    _img = keras.preprocessing.image.load_img(f'tub_1_22-11-12/images/{filename}')
    img = keras.preprocessing.image.img_to_array(_img)
    images[count,:] = img
    

In [ ]:
for block in tqdm(range(0, int(len(image_files)/1000))):
    # Normalize. Same was done during training.
    images2 = np.copy(images[1000*block:1000*(block+1)]/255)
    
    to_modify = np.ones(1000, dtype="int8")
    # The target for the image. This will be full angle in opposite direction
    true_angles = np.copy(true_angles_all[1000*block:1000*(block+1)])
    targets = np.ones((1000,1), dtype="int8")
    targets[true_angles > 0] = -1
    targets = tf.convert_to_tensor(targets, dtype="float")

    # Get the adversarial image
    # Find the target.This opposite steering angle

    for i in range(15):
        print(f'{i} iterations done')
        if np.sum(to_modify) == 0:
            break
        with tf.GradientTape() as tape:
            imgs_tf = tf.Variable(images2[to_modify == 1], dtype=float)
            outputs = model(imgs_tf, training=False)
            loss = (outputs[0]-targets[to_modify == 1])**2 #MSE

        grads = tape.gradient(loss, imgs_tf)
        grads /= (np.sqrt(np.sum(grads**2)))

        images2[to_modify == 1] -= lr*np.array(grads)
        
        # We modify until the difference between original value and the prediction is greater than 0.8
        to_modify[to_modify == 1][(np.abs(outputs[0]-true_angles[to_modify == 1]) >= 0.8).reshape(-1)] = 0
        
    images2[images2 < 0] = 0 # Keep values in range.
    images2[images2 > 1] = 1
    
    for i in range(1000):
        tf.keras.preprocessing.image.save_img(f'images_adversarial/tub_1_22-11-12_adversarial/images/{block*1000+i}_cam_image_array_.jpg', images2[i,:,:,:])